## Neural Networks

`torch.nn` is the package to contruct neural network in PyTorch.

`nn` depends on `autograd` to define models and differentiate them.

`nn.Module` contains layers and `forward(input)` method that returns output.

<img src="img/mnist_nn_pytorch.png">

It is a simple feed forwad network which takes images input, feeds it through several layers one after other and finally gives output.

A typical training procedure for a neural network is as follows:

1. Define the neural network that has some learnable parameters (or weights)
1. Iterate over a dataset of inputs
1. Process input through the network
1. Compute the loss (how far is the output from being correct)
1. Propagate gradients back into the network’s parameters
1. Update the weights of the network, typically using a simple update rule:

    `weight = weight - learning_rate * gradient`


### Define the network

In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution kernel
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self,x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d (1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d (6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120)
  (fc2): Linear(in_features=120, out_features=84)
  (fc3): Linear(in_features=84, out_features=10)
)


You just have to define the `forward` function, and the `backward` function (where gradients are computed) is automatically defined for you using `autograd`. You can use any of the Tensor operations in the `forward` function.

The learnable parameters of a model are returned by `nn.parameters()`

In [2]:
param = list(net.parameters())
print(len(param))
print(param[0].size())

10
torch.Size([6, 1, 5, 5])


The input to the `forward` is `autograd.Variable` and so the output is.

Expected input size is 32x32.

In [3]:
input = Variable(torch.randn(1,1,32,32))
output = net(input)
print(output)

Variable containing:
1.00000e-02 *
  1.3145  6.5109  5.3714  4.1201  9.9175  7.8367 -4.6468 -1.6072  4.8334 -3.1832
[torch.FloatTensor of size 1x10]



In [4]:
net.zero_grad() #Sets gradients of all model parameters to zero.
output.backward(torch.randn(1,10))

Recap:

`torch.Tensor` - A multi-dimensional array.

`autograd.Variable` - Wraps a Tensor and records the history of operations applied to it. Has the same API as a `Tensor`, with some additions like `backward()`. Also holds the gradient w.r.t. the tensor.

`nn.Module` - Neural network module. Convenient way of encapsulating parameters, with helpers for moving them to GPU, exporting, loading, etc.

`nn.Parameter` - A kind of `Variable`, that is automatically registered as a parameter when assigned as an attribute to a `Module`.

`autograd.Function` - Implements `forward` and `backward` definitions of an autograd operation. Every `Variable` operation, creates at least a single `Function` node, that connects to functions that created a `Variable` and encodes its history.

### Loss Function

Loss function estimates how far away the output is from target. We are using `nn.MSELoss` (to compute mean-squared error between output and target)

In [5]:
target = Variable(torch.arange(1,11)) # dummy target 1 to 10 numbers
loss_fn = nn.MSELoss()

loss = loss_fn(output, target)
print(loss)

Variable containing:
 38.2834
[torch.FloatTensor of size 1]



In [6]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

if you follow `loss` in the backward direction, using it’s `.grad_fn` attribute, you will see a graph of computations that looks like this:

input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss

### Backprop

In [7]:
net.zero_grad() # clear existing gradients of all parameters, 
                # else gradients will be accumulated to existing gradients

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward(retain_graph=True) # backpropagate the error 
                # The whole graph is differentiated w.r.t. the loss, 
                # and all Variables in the graph will have their .grad Variable accumulated with the gradient.

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]



RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

### Update the weights

The simplest update rule used in practice is the Stochastic Gradient Descent (SGD):

    `weight = weight - learning_rate * gradient`

We can implement this using simple python code:

In [8]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

However there is `torch.optim` package which has various update rules like SGD, Nesterov-SGD, Adam, RMSProp, etc.  

In [9]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = loss_fn(output, target)
loss.backward()
optimizer.step()    # Does the update
